In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
import glob

<IPython.core.display.Javascript object>

In [3]:
!ls all_cements/grouped

BidirectionalGRU_results_grouped_1.csv	 lstm_results_grouped_1.csv
BidirectionalGRU_results_grouped_2.csv	 lstm_results_grouped_2.csv
BidirectionalGRU_results_grouped_3.csv	 lstm_results_grouped_3.csv
BidirectionalGRU_results_grouped_4.csv	 lstm_results_grouped_4.csv
BidirectionalGRU_results_grouped_5.csv	 lstm_results_grouped_5.csv
BidirectionalGRU_results_grouped_6.csv	 lstm_results_grouped_6.csv
BidirectionalLSTM_results_grouped_1.csv  mlp_results_grouped_1.csv
BidirectionalLSTM_results_grouped_2.csv  mlp_results_grouped_2.csv
BidirectionalLSTM_results_grouped_3.csv  mlp_results_grouped_3.csv
BidirectionalLSTM_results_grouped_4.csv  mlp_results_grouped_4.csv
BidirectionalLSTM_results_grouped_5.csv  mlp_results_grouped_5.csv
BidirectionalLSTM_results_grouped_6.csv  mlp_results_grouped_6.csv
decision_tree_results_grouped_1.csv	 random_forest_results_grouped_1.csv
decision_tree_results_grouped_2.csv	 random_forest_results_grouped_2.csv
decision_tree_results_grouped_3.csv	 random_forest

<IPython.core.display.Javascript object>

### Reading The files

In [4]:
path = "all_cements/grouped/"
csv_files = glob.glob(os.path.join(path, "*.csv"))

<IPython.core.display.Javascript object>

In [5]:
results = []

for filepath in csv_files:
    df = pd.read_csv(filepath, header=[0, 1])
    results.append(df)
df = pd.concat(results, ignore_index=True)

<IPython.core.display.Javascript object>

### Renaming the Columns

In [6]:
df.columns = pd.MultiIndex.from_tuples(
    list(
        {
            col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
        }.values()
    )
)

<IPython.core.display.Javascript object>

### Defining the Columns to work with

In [7]:
cols = [
    ("Model", ""),
    ("Features", ""),
    ("Cross Validation", ""),
    ("Cross Validation Params", ""),
    ("RMSE Test", "mean"),
    ("RMSE Test", "std"),
    ("MAE Test", "mean"),
    ("MAE Test", "std"),
    ("MAPE Test", "mean"),
    ("MAPE Test", "std"),
    ("R2 Test", "mean"),
    ("R2 Test", "std"),
]

<IPython.core.display.Javascript object>

In [8]:
cols_metrics = [
    ("RMSE Test", "mean"),
    ("RMSE Test", "std"),
    ("MAE Test", "mean"),
    ("MAE Test", "std"),
    ("MAPE Test", "mean"),
    ("MAPE Test", "std"),
    ("R2 Test", "mean"),
    ("R2 Test", "std"),
]

<IPython.core.display.Javascript object>

# Linear Regression

In [9]:
df_linreg = df[df[("Model", "")] == "Linear Regression"][cols].copy()
df_linreg = df_linreg[df_linreg["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [10]:
df_linreg[df_linreg["Features"] == "Chemical + Mineralogical"]

Model                  Features            Cross Validation  \
                                                                               
174  Linear Regression  Chemical + Mineralogical  Blocking Time Series Split   
176  Linear Regression  Chemical + Mineralogical              Repeated KFold   
177  Linear Regression  Chemical + Mineralogical           Time Series Split   

                              Cross Validation Params RMSE Test            \
                                                           mean       std   
174  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.876559  0.462525   
176                     {"N_Splits": 5, "Repeats": 3}  3.053322  0.144758   
177           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.576885  0.363895   

     MAE Test           MAPE Test             R2 Test            
         mean       std      mean       std      mean       std  
174  2.872191  0.594589  0.059495  0.011507  0.596115  0.115605  
176  2.373273  0.091372  0.049463  0.002277  0.785187  0.020993  
177  2.740066  0.280383  0.056762  0.006759  0.689686  0.074950

<IPython.core.display.Javascript object>

In [11]:
idx_min = df_linreg[df_linreg["Features"] == "Chemical + Mineralogical"][
    "RMSE Test", "mean"
].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model                  Features Cross Validation  \
                                                                    
176  Linear Regression  Chemical + Mineralogical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
176  {"N_Splits": 5, "Repeats": 3}     3.053  0.145    2.373  0.091     0.049   

           R2 Test         
       std    mean    std  
176  0.002   0.785  0.021

<IPython.core.display.Javascript object>

In [12]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ Mineralogical \+ CS*"
pattern2 = r"Chemical \+ Mineralogical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

features_filter = df_linreg["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_linreg[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_linreg.loc[[idx_min]].round(3)

/tmp/ipykernel_2122092/383223557.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_linreg["Features"].str.contains(


Model                             Features Cross Validation  \
                                                                               
268  Linear Regression  Chemical + Mineralogical + Physical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
268  {"N_Splits": 5, "Repeats": 3}     2.289  0.099    1.773  0.067     0.037   

           R2 Test        
       std    mean   std  
268  0.001   0.879  0.01

<IPython.core.display.Javascript object>

In [13]:
idx_min = df_linreg[df_linreg["Features"] == "Chemical + Mineralogical + One-Hot"][
    "RMSE Test", "mean"
].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model                            Features Cross Validation  \
                                                                              
248  Linear Regression  Chemical + Mineralogical + One-Hot   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
248  {"N_Splits": 5, "Repeats": 3}     2.503  0.088    2.026  0.072     0.042   

           R2 Test         
       std    mean    std  
248  0.002   0.856  0.011

<IPython.core.display.Javascript object>

In [14]:
pattern = r"One-Hot"

features_filter = df_linreg["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_linreg[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model                                       Features  \
                                                                        
336  Linear Regression  Chemical + Mineralogical + Physical + One-Hot   

    Cross Validation        Cross Validation Params RMSE Test        MAE Test  \
                                                         mean    std     mean   
336   Repeated KFold  {"N_Splits": 5, "Repeats": 3}     2.182  0.084    1.704   

           MAPE Test        R2 Test         
       std      mean    std    mean    std  
336  0.059     0.035  0.001    0.89  0.008

<IPython.core.display.Javascript object>

# Neural Networks

In [15]:
neural_networks = [
    "MLP",
    "LSTM",
    "GRU",
    "BidirectionalGRU",
    "BidirectionalLSTM",
    "Transformer",
]

df_nn = df[df[("Model", "")].isin(neural_networks)][cols].copy()
# df_nn = df_nn.drop([("Model", "")], axis=1)
df_nn = df_nn[df_nn["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [26]:
idx_min = df_nn[df_nn["Features"] == "Chemical + Mineralogical"][
    "RMSE Test", "mean"
].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                  Features Cross Validation  \
                                                                  
58  BidirectionalGRU  Chemical + Mineralogical   Repeated KFold   

          Cross Validation Params RMSE Test        MAE Test       MAPE Test  \
                                       mean    std     mean   std      mean   
58  {"N_Splits": 5, "Repeats": 3}     2.801  0.177    2.154  0.14     0.045   

          R2 Test         
      std    mean    std  
58  0.003   0.819  0.023

<IPython.core.display.Javascript object>

In [17]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ Mineralogical \+ CS*"
pattern2 = r"Chemical \+ Mineralogical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

# x = df_nn["Features"] != "Chemical + Mineralogical + Physical"
features_filter = df_nn["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_nn[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_nn.loc[[idx_min]].round(3)

/tmp/ipykernel_2122092/1917742884.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_nn["Features"].str.contains(


Model                        Features Cross Validation  \
                                                                        
99  BidirectionalGRU  Chemical + Mineralogical + CS2   Repeated KFold   

          Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                       mean    std     mean    std      mean   
99  {"N_Splits": 5, "Repeats": 3}     2.129  0.124    1.648  0.085     0.034   

          R2 Test         
      std    mean    std  
99  0.002   0.895  0.013

<IPython.core.display.Javascript object>

In [18]:
idx_min = df_nn[df_nn["Features"] == "Chemical + Mineralogical + One-Hot"][
    "RMSE Test", "mean"
].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                            Features Cross Validation  \
                                                                            
47  BidirectionalGRU  Chemical + Mineralogical + One-Hot   Repeated KFold   

          Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                       mean    std     mean    std      mean   
47  {"N_Splits": 5, "Repeats": 3}     2.447  0.105    1.917  0.092     0.039   

          R2 Test         
      std    mean    std  
47  0.002   0.862  0.014

<IPython.core.display.Javascript object>

In [19]:
pattern = r"One-Hot"

features_filter = df_nn["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_nn[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                                  Features  \
                                                                  
110  BidirectionalGRU  Chemical + Mineralogical + CS2 + One-Hot   

    Cross Validation        Cross Validation Params RMSE Test       MAE Test  \
                                                         mean   std     mean   
110   Repeated KFold  {"N_Splits": 5, "Repeats": 3}     2.143  0.12    1.658   

           MAPE Test        R2 Test         
       std      mean    std    mean    std  
110  0.102     0.034  0.002   0.894  0.012

<IPython.core.display.Javascript object>

# Trees

In [20]:
trees = ["Decision Tree", "Random Forest", "XGBoost"]

df_trees = df[df[("Model", "")].isin(trees)][cols].copy()
df_trees = df_trees[df_trees["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [21]:
idx_min = df_trees[df_trees["Features"] == "Chemical + Mineralogical"][
    "RMSE Test", "mean"
].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                  Features Cross Validation  \
                                                          
272  XGBoost  Chemical + Mineralogical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
272  {"N_Splits": 5, "Repeats": 3}     3.164  0.145    2.509  0.119     0.053   

           R2 Test         
       std    mean    std  
272  0.003    0.77  0.019

<IPython.core.display.Javascript object>

In [22]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ Mineralogical \+ CS*"
pattern2 = r"Chemical \+ Mineralogical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

# x = df_nn["Features"] != "Chemical + Mineralogical + Physical"
features_filter = df_trees["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_trees[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_trees.loc[[idx_min]].round(3)

/tmp/ipykernel_2122092/2477923481.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_trees["Features"].str.contains(


Model                             Features Cross Validation  \
                                                                     
295  XGBoost  Chemical + Mineralogical + Physical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
295  {"N_Splits": 5, "Repeats": 3}     2.686  0.133    2.129  0.081     0.044   

           R2 Test         
       std    mean    std  
295  0.002   0.834  0.016

<IPython.core.display.Javascript object>

In [23]:
idx_min = df_trees[df_trees["Features"] == "Chemical + Mineralogical + One-Hot"][
    "RMSE Test", "mean"
].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                            Features Cross Validation  \
                                                                          
244  Random Forest  Chemical + Mineralogical + One-Hot   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
244  {"N_Splits": 5, "Repeats": 3}     2.626  0.111    2.131  0.088     0.044   

           R2 Test         
       std    mean    std  
244  0.002   0.841  0.015

<IPython.core.display.Javascript object>

In [24]:
pattern = r"One-Hot"

features_filter = df_trees["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_trees[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                                  Features Cross Validation  \
                                                                                
256  Random Forest  Chemical + Mineralogical + CS2 + One-Hot   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
256  {"N_Splits": 5, "Repeats": 3}     2.457  0.113    1.941  0.082      0.04   

           R2 Test         
       std    mean    std  
256  0.002   0.861  0.016

<IPython.core.display.Javascript object>